In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Training on {device}')

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                               download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                               download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse',
           'ship', 'truck')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64*4*4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=self.pool(F.relu(self.conv3(x)))
        x=x.view(-1, 64*4*4)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x
net=Net().to(device)
print(net)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9,
                      weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
def train_model():
    num_epochs = 50
    epoch_loss=[]
    for epoch in range(num_epochs):
        running_loss = 0.0
        all_batch_losses_in_epoch = []
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs= net(inputs)
            loss=criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            all_batch_losses_in_epoch.append(loss.item())
            if i % 200 ==199:
                print(f'[Epoch {epoch+1}, Batch {i+1:5d}]\
                 loss: {running_loss/200:.3f}')
                running_loss=0.0
        avg_epoch_loss = sum(all_batch_losses_in_epoch) /\
         len(all_batch_losses_in_epoch)
        epoch_loss.append(avg_epoch_loss)
        scheduler.step()
    print('Finished Training')
    return net, epoch_loss

In [ ]:
if __name__ == '__main__':
    net, epoch_loss = train_model()

In [ ]:
plt.figure()
plt.plot(epoch_loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted ==labels).sum().item()
print(f'Accuracy of the network on the 10000 test images:\
 {100* correct/total:.2f} %')